In [50]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie

In [84]:
function TM(u, p, t)

    U(y) = U0 + ΔU0 / ( 1 + exp( -50*(y-ythr) ) )
    σ(x) = 1 / ( 1 + exp( -20*(x-xthr) ) )

    E, x, y = u
    α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0 = p

    du1 = (-E + α * log( 1 + exp( (J * U(y) * x * E + I0  ) / (α) ) ) ) / τ
    du2 = (1-x)/τD - U(y)*x*E
    du3 = (-y)/τy+β*σ(x)

    return SVector(du1, du2, du3)
end

TM (generic function with 1 method)

In [134]:
t = 100.0
tt = 100.0
tstep = 0.001
trange = range(0.0, t, step = tstep);
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [135]:
const τ = 0.013; const τD = 0.15; const τy = 1.8; const J = 3.07; const β = 0.4375
const xthr = 0.9; const ythr = 0.4
α = 0.0900; U0 = 0.40; ΔU0 = 0.330; I0 =  -0.09635;

In [136]:
u0 = [0.5016565343933567
 0.8843561571148351
 0.3465491964077342]
p = [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
ds = ContinuousDynamicalSystem(TM, u0, p)

3-dimensional continuous dynamical system
 state:       [0.501657, 0.884356, 0.346549]
 rule f:      TM
 in-place?    false
 jacobian:    ForwardDiff
 parameters:  [0.09, 0.013, 0.15, 1.8, 3.07, …, 0.4, 0.33, 0.4375, -0.09635]

In [137]:
I0range = range( -0.09639, -0.0969, step = -0.0000002 ) # -0.0000004

idx_control_parameter = 11

idx_fix = 1; fixed_value = 20.0
surface = (idx_fix, fixed_value)
setting_root = (xrtol = 1e-11, atol = 1e-11);

In [138]:
length(I0range)

2551

In [ ]:
idx_save = 2
output = produce_orbitdiagram(ds, surface, idx_save,
                        idx_control_parameter, I0range;
                        tfinal = t, Ttr = tt,
                        printparams = true,
                        diffeq = integ_set, rootkw = setting_root
                        );

parameter = -0.09639
parameter = -0.0963902
parameter = -0.0963904
parameter = -0.0963906
parameter = -0.0963908
parameter = -0.096391
parameter = -0.0963912
parameter = -0.0963914
parameter = -0.0963916
parameter = -0.0963918
parameter = -0.096392
parameter = -0.0963922
parameter = -0.0963924
parameter = -0.0963926
parameter = -0.0963928
parameter = -0.096393
parameter = -0.0963932
parameter = -0.0963934
parameter = -0.0963936
parameter = -0.0963938
parameter = -0.096394
parameter = -0.0963942
parameter = -0.0963944
parameter = -0.0963946
parameter = -0.0963948
parameter = -0.096395
parameter = -0.0963952
parameter = -0.0963954
parameter = -0.0963956
parameter = -0.0963958
parameter = -0.096396
parameter = -0.0963962
parameter = -0.0963964
parameter = -0.0963966
parameter = -0.0963968
parameter = -0.096397
parameter = -0.0963972
parameter = -0.0963974
parameter = -0.0963976
parameter = -0.0963978
parameter = -0.096398
parameter = -0.0963982
parameter = -0.0963984
parameter = -0.096398

parameter = -0.0964618
parameter = -0.096462
parameter = -0.0964622
parameter = -0.0964624
parameter = -0.0964626
parameter = -0.0964628
parameter = -0.096463
parameter = -0.0964632
parameter = -0.0964634
parameter = -0.0964636
parameter = -0.0964638
parameter = -0.096464
parameter = -0.0964642
parameter = -0.0964644
parameter = -0.0964646
parameter = -0.0964648
parameter = -0.096465
parameter = -0.0964652
parameter = -0.0964654
parameter = -0.0964656
parameter = -0.0964658
parameter = -0.096466
parameter = -0.0964662
parameter = -0.0964664
parameter = -0.0964666
parameter = -0.0964668
parameter = -0.096467
parameter = -0.0964672
parameter = -0.0964674
parameter = -0.0964676
parameter = -0.0964678
parameter = -0.096468
parameter = -0.0964682
parameter = -0.0964684
parameter = -0.0964686
parameter = -0.0964688
parameter = -0.096469
parameter = -0.0964692
parameter = -0.0964694
parameter = -0.0964696
parameter = -0.0964698
parameter = -0.09647
parameter = -0.0964702
parameter = -0.096470

parameter = -0.0965336
parameter = -0.0965338
parameter = -0.096534
parameter = -0.0965342
parameter = -0.0965344
parameter = -0.0965346
parameter = -0.0965348
parameter = -0.096535
parameter = -0.0965352
parameter = -0.0965354
parameter = -0.0965356
parameter = -0.0965358
parameter = -0.096536
parameter = -0.0965362
parameter = -0.0965364
parameter = -0.0965366
parameter = -0.0965368
parameter = -0.096537
parameter = -0.0965372
parameter = -0.0965374
parameter = -0.0965376
parameter = -0.0965378
parameter = -0.096538
parameter = -0.0965382
parameter = -0.0965384
parameter = -0.0965386
parameter = -0.0965388
parameter = -0.096539
parameter = -0.0965392
parameter = -0.0965394
parameter = -0.0965396
parameter = -0.0965398
parameter = -0.09654
parameter = -0.0965402
parameter = -0.0965404
parameter = -0.0965406
parameter = -0.0965408
parameter = -0.096541
parameter = -0.0965412
parameter = -0.0965414
parameter = -0.0965416
parameter = -0.0965418
parameter = -0.096542
parameter = -0.096542

┌ Warning: For parameter -0.0965956 The Poincaré surface of section did not have any points!
└ @ ChaosTools C:\Users\Alex\.julia\packages\ChaosTools\PHPDF\src\orbitdiagrams\produce_orbitdiagram.jl:75



parameter = -0.096596

...

┌ Warning: For parameter -0.096896 The Poincaré surface of section did not have any points!
└ @ ChaosTools C:\Users\Alex\.julia\packages\ChaosTools\PHPDF\src\orbitdiagrams\produce_orbitdiagram.jl:75


parameter = -0.0968964

┌ Warning: For parameter -0.0968962 The Poincaré surface of section did not have any points!
└ @ ChaosTools C:\Users\Alex\.julia\packages\ChaosTools\PHPDF\src\orbitdiagrams\produce_orbitdiagram.jl:75



parameter = -0.0968966

┌ Warning: For parameter -0.0968964 The Poincaré surface of section did not have any points!
└ @ ChaosTools C:\Users\Alex\.julia\packages\ChaosTools\PHPDF\src\orbitdiagrams\produce_orbitdiagram.jl:75


In [ ]:
f = Figure(resolution = (1200, 700))
axis = Axis(f[1,1]; xlabel = L"I_0", ylabel = L"X")
for (j, p) in enumerate(I0range)
    scatter!(axis, fill(p, length(output[j])), output[j]; color = ("blue", 1.0), markersize = 2.0)
end
f